In [1]:
import numpy as np
import os
import random

import tensorflow as tf
# sess = tf.InteractiveSession()
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.contrib import layers, rnn, seq2seq
from tqdm import tqdm

In [2]:
from data_generator import NLUDataGenerator
from tensorflow.contrib.rnn import LSTMCell, LSTMStateTuple
from tensorflow.contrib.legacy_seq2seq import rnn_decoder

In [3]:
PAD = 0
EOS = 1
MASK1_CLASSES = 10
MASK2_CLASSES = 10


TimeMajor = False
batch_size = 32
encoder_max_time = 64
DataGen = NLUDataGenerator('../data/usr_df_final.csv',
                           '../data/ontology_dstc2.json',
                           '../data/slots.txt', seq_len = encoder_max_time,
                           batch_size = batch_size, time_major=TimeMajor)
vocab_size = len(DataGen.vocab)
input_embedding_size = 128
encoder_hidden_units = 128
# decoder_hidden_units = encoder_hidden_units * 2


In [4]:
slots_number = len(DataGen.slots_encode)
acts_number = len(DataGen.acts_encode)
print("slots_number: ", slots_number)
print("acts_number: ", acts_number)
print("vocab_size: ", DataGen.vocab_size)

slots_number:  27
acts_number:  21
vocab_size:  824


In [5]:
tf.reset_default_graph()

inputs = tf.placeholder(shape=(None, None), dtype=tf.int32)
mask1 = tf.placeholder(shape=(None, None), dtype=tf.int32)
mask2 = tf.placeholder(shape=(None, None), dtype=tf.int32)
lens = tf.placeholder(shape=(None,), dtype=tf.int32)

embeddings = tf.get_variable('embedings', [vocab_size, input_embedding_size], tf.float32)

encoded = tf.nn.embedding_lookup(embeddings, inputs)
cell = LSTMCell(input_embedding_size)

outputs, _ = tf.nn.dynamic_rnn(cell, encoded, sequence_length=lens, dtype=tf.float32)


seq_mask = tf.to_float(tf.greater(inputs, PAD))
with tf.variable_scope('lm-reg'):
    first_words = tf.slice(inputs, [0, 1], [-1, -1])
    target = tf.pad(first_words, [[0, 0], [0, 1]])
    nwp = layers.fully_connected(outputs, vocab_size, activation_fn=None)
    lm_loss = seq2seq.sequence_loss(nwp, target, seq_mask)

with tf.variable_scope('mask-pred'):
    m1p = layers.fully_connected(outputs, slots_number, activation_fn=None)
    m2p = layers.fully_connected(outputs, acts_number, activation_fn=None)
    m1_loss = seq2seq.sequence_loss(m1p, mask1, seq_mask)
    m2_loss = seq2seq.sequence_loss(m2p, mask2, seq_mask)
    
    
lr = tf.Variable(0.01, trainable=False)
opt = tf.train.GradientDescentOptimizer(lr)
train_op = opt.minimize(lm_loss + m1_loss + m2_loss)

In [6]:
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    
    avg_loss = -np.ones((3, ))
    alpha = 0.9
    
    pbar = tqdm(range(500))
    for step in pbar:
        x, m1, m2, _lens = next(DataGen)
        
        res = sess.run([lm_loss, m1_loss, m2_loss, train_op], {inputs: x, mask1: m1, mask2: m2, lens: _lens})
        res = np.array(res[:-1])
        
        if avg_loss[0] < 0:
            avg_loss = res
        
        avg_loss = alpha * avg_loss + (1-alpha) * res
        
        pbar.set_description(' '.join('{:.2f}'.format(_) for _ in avg_loss))

Instructions for updating:
Use `tf.global_variables_initializer` instead.


5.89 1.35 1.15: 100%|██████████| 500/500 [00:13<00:00, 36.98it/s]


In [ ]:
file_writer = tf.summary.FileWriter('../logdir/', sess.graph)

## Это чисто черновик, где происходит какая-то фигня с bi-LSTM

In [ ]:
encoder_inputs = tf.placeholder(shape=(None, None), dtype=tf.int32, name='encoder_inputs')
encoder_inputs_length = tf.placeholder(shape=(None,), dtype=tf.int32, name='encoder_inputs_length')

encoder_targets = tf.placeholder(shape=(None), dtype=tf.int32, name='encoder_targets')
embeddings = tf.Variable(tf.random_uniform([vocab_size, input_embedding_size], -1.0, 1.0), dtype=tf.float32)

encoder_inputs_embedded = tf.nn.embedding_lookup(embeddings, encoder_inputs)

In [ ]:
encoder_cell_b = LSTMCell(encoder_hidden_units)
encoder_cell_f = LSTMCell(encoder_hidden_units, reuse=None)

In [ ]:
# encoder_cell = LSTMCell(encoder_hidden_units, reuse=tf.get_variable_scope().reuse)
# encoder_cell_f = LSTMCell(encoder_hidden_units)
((encoder_fw_outputs,
encoder_bw_outputs),
(encoder_fw_final_state,
encoder_bw_final_state)) = (tf.nn.bidirectional_dynamic_rnn(cell_fw=encoder_cell_f,
                                cell_bw=encoder_cell_b,
                                inputs=encoder_inputs_embedded,
                                sequence_length=encoder_inputs_length,
                                dtype=tf.float32, time_major=TimeMajor))
    

In [ ]:
encoder_outputs = tf.concat((encoder_fw_outputs, encoder_bw_outputs), 2)

encoder_final_state_c = tf.concat(
    (encoder_fw_final_state.c, encoder_bw_final_state.c), 1)

encoder_final_state_h = tf.concat(
    (encoder_fw_final_state.h, encoder_bw_final_state.h), 1)

encoder_final_state = LSTMStateTuple(
    c=encoder_final_state_c,
    h=encoder_final_state_h)

In [ ]:
from tensorflow.contrib import layers
next_word_pred = layers.fully_connected(encoder_outputs, vocab_size, ...)



mask1_pred = layers.fully_connected(encoder_outputs, mask1_vocab, ...)
mask2_pred = layers.fully_connected(encoder_outputs, mask2_vocab, ...)


In [ ]:
encoder_final_state_h

In [ ]:
W = tf.Variable(tf.random_uniform([encoder_hidden_units*2, vocab_size], -1, 1), dtype=tf.float32)
b = tf.Variable(tf.zeros([vocab_size]), dtype=tf.float32)

In [ ]:
logits = tf.add(tf.matmul(encoder_final_state_h, W), b)
# encoder_logits = tf.reshape(encoder_logits_flat, (decoder_max_steps, decoder_batch_size, vocab_size))

In [ ]:
logits

In [ ]:
encoder_targets

In [ ]:
losses = tf.nn.sparse_softmax_cross_entropy_with_logits(logits = logits, labels = encoder_targets)